In [22]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt

import json
from skimage.transform import resize
import tensorflow as tf
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [29]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data

img_generator = ImageDataGenerator(
    rescale=1./255.,
    samplewise_center=True,
    samplewise_std_normalization=True,
    preprocessing_function=preprocess_input
)

def check_dicom(filename):
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)       
    img = ds.pixel_array
    return img

    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img, img_size):
    
    #proc_img = normalize_meanstd(img)
    proc_img = img_generator.apply_transform(img, {})
    proc_img = resize(proc_img, img_size)
    
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    
    with open('my_model.json') as json_file:
        my_model = json.load(json_file)
        
    loaded_model = tf.keras.models.model_from_json(my_model)
    loaded_model.load_weights(weight_path)
    
    return loaded_model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, threshold): 
    prediction = model.predict(img)
    print(prediction[0][0])
    
    if prediction[0][0] >= threshold:
        return "Pneumonia"
    return "No pneumonia"

In [30]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = "./models/my_model.h5"
weight_path = "xray_class_my_model.best.hdf5"

IMG_SIZE=(1,224,224,3) # Might be different depending on used architecture

my_model = load_model(model_path, weight_path)

for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img, IMG_SIZE)
    pred = predict_image(my_model, img_proc, 0.6)
    print(pred)

Load file test1.dcm ...
0.61092764
Pneumonia
Load file test2.dcm ...
0.5635268
No pneumonia
Load file test3.dcm ...
0.55672526
No pneumonia
Load file test4.dcm ...
0.61092764
Pneumonia
Load file test5.dcm ...
0.61092764
Pneumonia
Load file test6.dcm ...
0.61092764
Pneumonia
